In [ ]:
import numpy as np
import pandas as pd
import tarfile
import urllib.request
from pathlib import Path

def load_housing_data():
    tarball_path = Path("numpyex/housing.tgz")
    if not tarball_path.is_file():
        Path("numpyex").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="numpyex")
    return pd.read_csv(Path("numpyex/housing/housing.csv"))
house=load_housing_data()
house.head()




In [ ]:
house.info()

In [ ]:

import matplotlib.pyplot as plt
plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
house.hist(bins=46,figsize=(12,10))
plt.show()




In [40]:
from sklearn.model_selection import train_test_split
import numpy as np
def suffle_split(data,test_ratio):
    suffle=np.random.permutation(len(house))
    test_size=int(len(data)*test_ratio)
    test_split=suffle[:test_size]
    train_split=suffle[test_size:]
    return (data.iloc[test_split] , data.iloc[train_split]).bool()






    


In [44]:
train_split,test_split=suffle_split(house,0.3)
print(len(train_split))

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().